In [ ]:
import scrapy
import pandas as pd

class PttSpider(scrapy.Spider):
    name = 'ptt_spider'
    allowed_domains = ['www.ptt.cc']
    start_urls = ['https://www.ptt.cc/bbs/AfterPhD/M.1246244269.A.F38.html']

    def parse(self, response):
        data_column_name = ["學校", "系所", "應徵人數"]
        df_list = []

        for page_num in range(1, 9999):  # assuming there are less than 9999 pages
            # Get data from the current page
            data = self.get_data(response)
            df_list.extend(data)

            # Check if this is the last page
            if self.is_last_page(response, page_num):
                self.log("This is the last page")
                break

            # Follow the next page link
            next_page_url = self.get_next_page_url(response)
            yield scrapy.Request(next_page_url, callback=self.parse)

        # Create DataFrame and save to CSV
        df = pd.DataFrame(df_list, columns=data_column_name)
        df.to_csv("教職應徵人數之統計.csv", index=False, encoding="utf-8")

    def get_data(self, response):
        data = []
        rows = response.css('table.csscts tr')
        
        for row in rows:
            cells = row.css('td.wide-only::text').extract()
            if len(cells) == 8:
                data.append(cells[1:])

        return data

    def is_last_page(self, response, current_page):
        total_page = int(response.css('#lbl_Allpage::text').get())
        return current_page >= total_page - 1

    def get_next_page_url(self, response):
        next_button = response.css('#ImageButton8::attr(onclick)').re_first(r"window.location='(.+)'")
        return response.urljoin(next_button)
